# Writing Instances: Apply

We assume that you have [generated a SDK](generation.html) for the `Windmill` model and have a client ready to go.

The SDK generated by `SDK` supports nested upsert

In [7]:
import warnings
warnings.filterwarnings('ignore')
# This is just to enable improting the generated SDK from the examples folder in the pygen repository
import sys
from tests.constants import REPO_ROOT
sys.path.append(str(REPO_ROOT / "examples" ))

In [8]:
from windmill.client import WindmillClient

In [9]:
client = WindmillClient.from_toml("config.toml")

## Constructing new Windmill

In [10]:
from cognite.client.data_classes import TimeSeries
from windmill.client.data_classes import WindmillApply, BladeApply, NacelleApply, RotorApply

Lets construct a new windmill with TimeSeries. Note the example below is not complete (some TimeSeries and components are missing), 
but is kept short to make it easier to grasp

In [18]:
new_windmill = WindmillApply(
    external_id="windmill:demo",
    capacity=10.0,
    windfarm="Fornebu",
    name="Windmill ATH",
    rotor=RotorApply(
        external_id="windmill:demo:rotor",
        rotor_speed_controller=TimeSeries(external_id="windmill:demo:rotor:speed_controller"),
        rpm_low_speed_shaft=TimeSeries(external_id="windmill:demo:rotor:rpm_low_speed_shaft"),
    ),
    nacelle=NacelleApply(
        external_id="windmill:demo:nacelle",
        acc_from_back_side_x=TimeSeries(external_id="windmill:demo:acc_from_back_side_x"),
        acc_from_back_side_y=TimeSeries(external_id="windmill:demo:acc_from_back_side_y"),
        acc_from_back_side_z=TimeSeries(external_id="windmill:demo:acc_from_back_side_z"),
    ),
    blades=[
        BladeApply(
            external_id="windmill:demo:blade1",
            is_damaged=False,
            name="Blade 1",
        ),
        BladeApply(
            external_id="windmill:demo:blade2",
            is_damaged=False,
            name="Blade 2",
        ),
        BladeApply(
            external_id="windmill:demo:blade3",
            is_damaged=True,
            name="Blade 3",
        ),
    ]
)

When writing nested data we can specify edges either with an external id for the end node, or another data class.

The advangage of using a nested data class is that we can express edges without being explicit. In the example above, we are able to express that the blades `Blade 1-3` are connected to the windmill `windmill ATH` and that the `windmill ATH` is also linked to a nacelle and rotor.

## Inspecting Resources to create

We can inspect the nodes, edges and other resources that will be created by using the `.to_instances_apply` on the new windmill object.

In [19]:
resources = new_windmill.to_instances_apply()

In [23]:
len(resources.nodes), len(resources.edges), len(resources.time_series)

(6, 3, 5)

In [20]:
resources.nodes

,instance_type,space,external_id,sources
0,node,windmill-instances,windmill:demo,"[{'properties': {'capacity': 10.0, 'nacelle': ..."
1,node,windmill-instances,windmill:demo:blade1,"[{'properties': {'is_damaged': False, 'name': ..."
2,node,windmill-instances,windmill:demo:blade2,"[{'properties': {'is_damaged': False, 'name': ..."
3,node,windmill-instances,windmill:demo:blade3,"[{'properties': {'is_damaged': True, 'name': '..."
4,node,windmill-instances,windmill:demo:nacelle,[{'properties': {'acc_from_back_side_x': 'wind...
5,node,windmill-instances,windmill:demo:rotor,[{'properties': {'rotor_speed_controller': 'wi...


In [21]:
resources.edges

,instance_type,space,external_id,type,start_node,end_node
0,edge,windmill-instances,windmill:demo:windmill:demo:blade1,"{'space': 'power-models', 'external_id': 'Wind...","{'space': 'windmill-instances', 'external_id':...","{'space': 'windmill-instances', 'external_id':..."
1,edge,windmill-instances,windmill:demo:windmill:demo:blade2,"{'space': 'power-models', 'external_id': 'Wind...","{'space': 'windmill-instances', 'external_id':...","{'space': 'windmill-instances', 'external_id':..."
2,edge,windmill-instances,windmill:demo:windmill:demo:blade3,"{'space': 'power-models', 'external_id': 'Wind...","{'space': 'windmill-instances', 'external_id':...","{'space': 'windmill-instances', 'external_id':..."


In [22]:
resources.time_series

,external_id
0,windmill:demo:acc_from_back_side_x
1,windmill:demo:acc_from_back_side_y
2,windmill:demo:acc_from_back_side_z
3,windmill:demo:rotor:speed_controller
4,windmill:demo:rotor:rpm_low_speed_shaft


## Creating new Windmill

In [24]:
created = client.windmill.apply(new_windmill)

Note that the call above created 6 nodes, 3 edges and 5 time series:

In [25]:
created.nodes

,instance_type,space,external_id,version,was_modified,last_updated_time,created_time
0,node,windmill-instances,windmill:demo,1,True,2023-12-30 15:06:10.973,2023-12-30 15:06:10.973
1,node,windmill-instances,windmill:demo:blade1,1,True,2023-12-30 15:06:10.973,2023-12-30 15:06:10.973
2,node,windmill-instances,windmill:demo:blade2,1,True,2023-12-30 15:06:10.973,2023-12-30 15:06:10.973
3,node,windmill-instances,windmill:demo:blade3,1,True,2023-12-30 15:06:10.973,2023-12-30 15:06:10.973
4,node,windmill-instances,windmill:demo:nacelle,2,True,2023-12-30 15:06:10.973,2023-12-30 15:06:10.973
5,node,windmill-instances,windmill:demo:rotor,2,True,2023-12-30 15:06:10.973,2023-12-30 15:06:10.973


In [26]:
created.edges

,instance_type,space,external_id,version,was_modified,last_updated_time,created_time
0,edge,windmill-instances,windmill:demo:windmill:demo:blade1,1,True,2023-12-30 15:06:10.973,2023-12-30 15:06:10.973
1,edge,windmill-instances,windmill:demo:windmill:demo:blade2,1,True,2023-12-30 15:06:10.973,2023-12-30 15:06:10.973
2,edge,windmill-instances,windmill:demo:windmill:demo:blade3,1,True,2023-12-30 15:06:10.973,2023-12-30 15:06:10.973


And we can now see that the movie is now listed among the others.

In [27]:
created.time_series

,id,external_id,is_string,metadata,is_step,security_categories,created_time,last_updated_time
0,8433717001424809,windmill:demo:acc_from_back_side_x,False,{},False,[],2023-12-30 15:06:11.279,2023-12-30 15:06:11.279
1,732635896066219,windmill:demo:acc_from_back_side_y,False,{},False,[],2023-12-30 15:06:11.279,2023-12-30 15:06:11.279
2,6785552778393385,windmill:demo:acc_from_back_side_z,False,{},False,[],2023-12-30 15:06:11.279,2023-12-30 15:06:11.279
3,1618952669202701,windmill:demo:rotor:speed_controller,False,{},False,[],2023-12-30 15:06:11.279,2023-12-30 15:06:11.279
4,6361443697764395,windmill:demo:rotor:rpm_low_speed_shaft,False,{},False,[],2023-12-30 15:06:11.279,2023-12-30 15:06:11.279


We can inspect the newly created windmill by calling retrieve with the external id

In [28]:
client.windmill.retrieve(new_windmill.external_id)

,value
external_id,windmill:demo
blades,"[windmill:demo:blade1, windmill:demo:blade2, w..."
capacity,10.0
metmast,None
nacelle,windmill:demo:nacelle
name,Windmill ATH
rotor,windmill:demo:rotor
windfarm,Fornebu


# Deleting Instances

You can delete by passing and external ID or a sequence of external id to the delete method.

We can delete the newly created windmill 

In [29]:
client.windmill.list()

,external_id,blades,capacity,metmast,nacelle,name,rotor,windfarm
0,hornsea_1_mill_3,"[blade:1, blade:2, blade:3]",7.0,None,nacelle:1,hornsea_1_mill_3,rotor:1,Hornsea 1
1,hornsea_1_mill_2,"[blade:4, blade:5, blade:6]",7.0,None,nacelle:2,hornsea_1_mill_2,rotor:2,Hornsea 1
2,hornsea_1_mill_1,"[blade:7, blade:8, blade:9]",7.0,None,nacelle:3,hornsea_1_mill_1,rotor:3,Hornsea 1
3,hornsea_1_mill_4,"[blade:10, blade:11, blade:12]",7.0,None,nacelle:4,hornsea_1_mill_4,rotor:4,Hornsea 1
4,hornsea_1_mill_5,"[blade:13, blade:14, blade:15]",7.0,None,nacelle:5,hornsea_1_mill_5,rotor:5,Hornsea 1
5,windmill:demo,"[windmill:demo:blade1, windmill:demo:blade2, w...",10.0,None,windmill:demo:nacelle,Windmill ATH,windmill:demo:rotor,Fornebu


In [31]:
client.windmill.delete("windmill:demo")

InstancesDeleteResult(nodes=[NodeId(space='windmill-instances', external_id='windmill:demo')], edges=[])

After the delete call the new windmill is gone

In [32]:
client.windmill.list()

,external_id,blades,capacity,metmast,nacelle,name,rotor,windfarm
0,hornsea_1_mill_3,"[blade:1, blade:2, blade:3]",7.0,None,nacelle:1,hornsea_1_mill_3,rotor:1,Hornsea 1
1,hornsea_1_mill_2,"[blade:4, blade:5, blade:6]",7.0,None,nacelle:2,hornsea_1_mill_2,rotor:2,Hornsea 1
2,hornsea_1_mill_1,"[blade:7, blade:8, blade:9]",7.0,None,nacelle:3,hornsea_1_mill_1,rotor:3,Hornsea 1
3,hornsea_1_mill_4,"[blade:10, blade:11, blade:12]",7.0,None,nacelle:4,hornsea_1_mill_4,rotor:4,Hornsea 1
4,hornsea_1_mill_5,"[blade:13, blade:14, blade:15]",7.0,None,nacelle:5,hornsea_1_mill_5,rotor:5,Hornsea 1


In [40]:
# Cleanup windmill and timeseries
cdf = client.windmill._client

cdf.data_modeling.instances.delete(nodes=created.nodes.as_ids(), edges=created.edges.as_ids());

cdf.time_series.delete(external_id=created.time_series.as_external_ids(), ignore_unknown_ids=True);